# Membangun dengan Model Mistral

## Pendahuluan

Pelajaran ini akan membahas:
- Menjelajahi berbagai Model Mistral
- Memahami kasus penggunaan dan skenario untuk setiap model
- Contoh kode yang menunjukkan fitur unik dari setiap model


## Model Mistral

Dalam pelajaran ini, kita akan membahas 3 model Mistral yang berbeda: **Mistral Large**, **Mistral Small**, dan **Mistral Nemo**.

Setiap model ini tersedia gratis di marketplace Model Github. Kode dalam notebook ini akan menggunakan model-model tersebut untuk menjalankan kode. Berikut informasi lebih lanjut tentang penggunaan Github Models untuk [prototyping dengan model AI](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst).


## Mistral Large 2 (2407)
Mistral Large 2 saat ini merupakan model andalan dari Mistral dan dirancang untuk penggunaan di tingkat perusahaan.

Model ini merupakan peningkatan dari Mistral Large versi sebelumnya dengan menawarkan
-  Jendela Konteks Lebih Besar - 128k dibanding 32k
-  Performa Lebih Baik pada Tugas Matematika dan Pemrograman - akurasi rata-rata 76,9% dibanding 60,4%
-  Peningkatan performa multibahasa - bahasa yang didukung meliputi: Inggris, Prancis, Jerman, Spanyol, Italia, Portugis, Belanda, Rusia, Mandarin, Jepang, Korea, Arab, dan Hindi.

Dengan fitur-fitur ini, Mistral Large unggul dalam
- *Retrieval Augmented Generation (RAG)* - berkat jendela konteks yang lebih besar
- *Function Calling* - model ini mendukung function calling secara native sehingga bisa terintegrasi dengan alat eksternal dan API. Pemanggilan ini bisa dilakukan secara paralel maupun berurutan satu per satu.
- *Code Generation* - model ini sangat baik dalam menghasilkan kode Python, Java, TypeScript, dan C++.


Dalam contoh ini, kita menggunakan Mistral Large 2 untuk menjalankan pola RAG pada sebuah dokumen teks. Pertanyaannya ditulis dalam bahasa Korea dan menanyakan tentang aktivitas penulis sebelum kuliah.

Model Cohere Embeddings digunakan untuk membuat embedding dari dokumen teks serta pertanyaannya. Untuk contoh ini, digunakan paket Python faiss sebagai penyimpanan vektor.

Prompt yang dikirim ke model Mistral mencakup pertanyaan dan potongan-potongan yang diambil yang mirip dengan pertanyaan tersebut. Model kemudian memberikan jawaban dalam bahasa alami.


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## Mistral Small
Mistral Small adalah salah satu model dalam keluarga Mistral yang masuk kategori premier/enterprise. Seperti namanya, model ini merupakan Small Language Model (SLM). Keunggulan menggunakan Mistral Small antara lain:
- Hemat biaya dibandingkan LLM Mistral seperti Mistral Large dan NeMo - penurunan harga hingga 80%
- Latensi rendah - respons lebih cepat dibandingkan LLM Mistral
- Fleksibel - bisa digunakan di berbagai lingkungan dengan persyaratan sumber daya yang lebih ringan.

Mistral Small sangat cocok untuk:
- Tugas berbasis teks seperti merangkum, analisis sentimen, dan terjemahan.
- Aplikasi yang membutuhkan permintaan sering karena lebih hemat biaya
- Tugas kode dengan latensi rendah seperti review dan saran kode


## Membandingkan Mistral Small dan Mistral Large

Untuk menunjukkan perbedaan latensi antara Mistral Small dan Large, jalankan sel di bawah ini.

Kamu seharusnya bisa melihat perbedaan waktu respons sekitar 3-5 detik. Perhatikan juga panjang dan gaya respons pada prompt yang sama.


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## Mistral NeMo

Dibandingkan dengan dua model lain yang dibahas dalam pelajaran ini, Mistral NeMo adalah satu-satunya model gratis dengan Lisensi Apache2.

Model ini dianggap sebagai peningkatan dari LLM open source sebelumnya dari Mistral, yaitu Mistral 7B.

Beberapa fitur lain dari model NeMo adalah:

- *Tokenisasi yang lebih efisien:* Model ini menggunakan tokenizer Tekken, bukan tiktoken yang lebih umum digunakan. Hal ini memungkinkan performa yang lebih baik untuk berbagai bahasa dan kode.

- *Finetuning:* Model dasar tersedia untuk finetuning. Ini memberikan fleksibilitas lebih untuk kasus penggunaan yang membutuhkan penyesuaian lebih lanjut.

- *Native Function Calling* - Seperti Mistral Large, model ini telah dilatih untuk function calling. Hal ini membuatnya unik sebagai salah satu model open source pertama yang memiliki kemampuan tersebut.


## Mistral NeMo

Dibandingkan dengan dua model lain yang dibahas dalam pelajaran ini, Mistral NeMo adalah satu-satunya model gratis dengan Lisensi Apache2.

Model ini dianggap sebagai peningkatan dari LLM open source sebelumnya dari Mistral, yaitu Mistral 7B.

Beberapa fitur lain dari model NeMo adalah:

- *Tokenisasi yang lebih efisien:* Model ini menggunakan tokenizer Tekken, bukan tiktoken yang lebih umum digunakan. Hal ini memungkinkan performa yang lebih baik untuk berbagai bahasa dan kode.

- *Finetuning:* Model dasar tersedia untuk finetuning. Ini memberikan fleksibilitas lebih untuk kasus penggunaan yang membutuhkan penyesuaian lebih lanjut.

- *Native Function Calling* - Seperti Mistral Large, model ini telah dilatih untuk function calling. Hal ini membuatnya unik sebagai salah satu model open source pertama yang memiliki kemampuan tersebut.


### Membandingkan Tokenizer

Dalam contoh ini, kita akan melihat bagaimana Mistral NeMo menangani tokenisasi dibandingkan dengan Mistral Large.

Kedua contoh menggunakan prompt yang sama, namun kamu akan melihat bahwa NeMo menghasilkan jumlah token yang lebih sedikit dibandingkan Mistral Large.


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## Belajar tidak berhenti di sini, lanjutkan Perjalananmu

Setelah menyelesaikan pelajaran ini, lihat [Koleksi Pembelajaran AI Generatif](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) kami untuk terus meningkatkan pengetahuanmu tentang AI Generatif!



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berupaya untuk memberikan terjemahan yang akurat, harap diketahui bahwa terjemahan otomatis dapat mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang berwenang. Untuk informasi yang bersifat kritis, disarankan menggunakan jasa terjemahan profesional oleh manusia. Kami tidak bertanggung jawab atas kesalahpahaman atau penafsiran yang timbul akibat penggunaan terjemahan ini.
